In [1]:
using BenchmarkTools
using Distances
using NearestNeighborDescent
using NearestNeighborDescent: brute_knn, brute_search

In [2]:
include("./benchutils.jl");

In [3]:
function bmark(name, data, queries, metric)
    print(name, "\n")
    print("Graph construction: ")
    knn_graph = @btime DescentGraph($data, 10, $metric)
    nn = getindex.(knn_graph.graph, 1)
    brute_graph = brute_knn(data, metric, 10)
    true_nn = getindex.(brute_graph, 1)
    @show recall(nn, true_nn)
    true_idx, true_dist = brute_search(data, queries, 10, metric)
    idx, dist = search(knn_graph, queries, 10, 70)
    @show recall(idx, true_idx)
    q_per_sec = length(queries)/(@belapsed search($knn_graph, $queries, 10, 70))
    @show q_per_sec
    return
end

bmark (generic function with 1 method)

In [4]:
bmark("Fashion MNIST", FMNIST_data, FMNIST_queries, Euclidean())

Fashion MNIST
Graph construction:   1.140 s (10434566 allocations: 506.50 MiB)
recall(nn, true_nn) = 0.9969600000000001
recall(idx, true_idx) = 0.908799999999999
q_per_sec = 302.60540029936294


In [5]:
bmark("MNIST", MNIST_data, MNIST_queries, Euclidean())

MNIST
Graph construction:   1.106 s (9981428 allocations: 506.69 MiB)
recall(nn, true_nn) = 0.9895400000000002
recall(idx, true_idx) = 0.9873999999999996
q_per_sec = 282.14214066102863


In [6]:
# Cosine Tests 
bmark("Cosine Random", rand_data, rand_queries, CosineDist())

Cosine Random
Graph construction:   1.658 s (20970290 allocations: 784.45 MiB)
recall(nn, true_nn) = 0.7455800000000049
recall(idx, true_idx) = 0.9293999999999979
q_per_sec = 273.9267928739514


In [7]:
# Hamming Tests
bmark("Hamming Random", ham_data, ham_queries, Hamming())

Hamming Random
Graph construction:   917.724 ms (6882332 allocations: 454.95 MiB)
recall(nn, true_nn) = 0.12828000000000445
recall(idx, true_idx) = 0.8247999999999999
q_per_sec = 195.56175415044817
